# Wikimedia data
Wikimedia Downloads: Analytics Datasets Info about Pageviews, mediacounts and unique devices:
Pageviews since may 2015:
https://dumps.wikimedia.org/other/pageviews/[YEAR]/[YEAR]-[2-DIGIT-MONTH]/projectviews-YYYYMMDD-HH0000

In [ ]:
# generate URLs for download
from datetime import datetime, timedelta
from itertools import chain
from functools import reduce
import requests
import re
import os

def time_range(init, end):
    while init<end:
        yield init
        init+=timedelta(hours=1)   
    
def explode_time(date):
    return date, {
        'year': date.year,
        'month': date.month,
        'day': date.day,
        'hour': date.hour,
    }

def gen_url(x):
    ( date, exploded_date ) = x
    ( url_templ ) = (
        'https://dumps.wikimedia.org/other/pageviews/'
        '{year}/{year}-{month:02d}/'
        'projectviews-{year}{month:02d}{day:02d}-{hour:02d}0000'
    )
    return date, url_templ.format(**exploded_date)

exploded_dates = map(explode_time, time_range(datetime(2016,10,31), datetime(2017,10,31)))
urls = map(gen_url, exploded_dates)

#list(urls)

def explode_data_lines(dateurl):
    ( date, url ) = dateurl
    
    file_name = 'cache/' + url.replace('/','_')
           
    if not os.path.exists(file_name):
        os.makedirs('cache', exist_ok=True)
        with open(file_name, 'w') as f:
            f.write(requests.get(url).text)
        f.closed
    
    with open(file_name, 'r') as f:
        file_content = f.read()
    f.closed

    for line in file_content.splitlines():
        r = re.compile(r'^(?P<project>.*) - (?P<hits>\d+)')
        m = r.match(line)
        yield {
            'date': date,
            'project': m.group('project'),
            'hits': m.group('hits'),
        }

def filter_wikipedia_en(data):
    return data['project'] == 'en'

data_generators = map(explode_data_lines, urls)

data_generator = reduce(chain, data_generators)

wikipedia_en_generator = filter(filter_wikipedia_en, data_generator)

for data in wikipedia_en_generator:
    print(data['date'], data['hits'])
